In [ ]:
from __future__ import print_function

from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
num_classes = 5
img_rows,img_cols = 48,48
batch_size = 8

train_data_dir = r"C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\train"
validation_data_dir = r"C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\validation"

train_datagen = ImageDataGenerator(
					rescale=1./255,
					rotation_range=30,
					shear_range=0.3,
					zoom_range=0.3,
					width_shift_range=0.4,
					height_shift_range=0.4,
					horizontal_flip=True,
					fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
					train_data_dir,
					color_mode='grayscale',
					target_size=(img_rows,img_cols),
					batch_size=batch_size,
					class_mode='categorical',
					shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
							validation_data_dir,
							color_mode='grayscale',
							target_size=(img_rows,img_cols),
							batch_size=batch_size,
							class_mode='categorical',
							shuffle=True)


model = Sequential()



In [ ]:
# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau



In [ ]:
checkpoint = ModelCheckpoint(r'C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\Emo_little-h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

nb_train_samples = 24176
nb_validation_samples = 3006
epochs=25

In [ ]:
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nbq_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)

In [ ]:
emoji={ "happy": 0,"angry": 1,"sad": 2, "neutral": 3,"surprised": 4,"lol": 5 }

In [ ]:
import cv2

In [5]:
#WITHOUT THE EMOTION WORKING
# while True:
#     # Grab a single frame of video
#     ret, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_classifier.detectMultiScale(gray, 1.3, 5)

#     for (x, y, w, h) in faces:
#         cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         roi_gray = gray[y:y + h, x:x + w]
#         roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
#         # rect,face,image = face_detector(frame)

#         if np.sum([roi_gray]) != 0:
#             roi = roi_gray.astype('float') / 255.0
#             roi = img_to_array(roi)
#             roi = np.expand_dims(roi, axis=0)

#             # make a prediction on the ROI, then lookup the class

#             preds = classifier.predict(roi)[0]
#             label = class_labels[preds.argmax()]
#             label_position = (x, y)
#             cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
#         else:
#             cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
#     cv2.imshow('Emotion Detector', frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [4]:
#### WORKING CODE
# while True:
#     # Grab a single frame of video
#     ret, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_classifier.detectMultiScale(gray, 1.3, 5)
#     img_copy = np.copy(frame)
    
#     for (x, y, w, h) in faces:
#         roi_gray = gray[y:y + h, x:x + w]
#         roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        
#         # rect,face,image = face_detector(frame)

#         if np.sum([roi_gray]) != 0:
#             roi = roi_gray.astype('float') / 255.0
#             roi = img_to_array(roi)
#             roi = np.expand_dims(roi, axis=0)

#             # make a prediction on the ROI, then lookup the class

#             preds = classifier.predict(roi)[0]
#             label = class_labels[preds.argmax()]
#             label_position = (x, y)
            
            
#             img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)
            
#             # Beard filter
#             santa_filter = cv2.imread(emoji_path, -1)
#             santa_filter = cv2.resize( santa_filter, (w,h))
#             sw,sh,sc = santa_filter.shape
            
#             for i in range(0,sw):       # Overlay the filter based on the alpha channel
#                 for j in range(0,sh):
#                     img_copy[i+y,j+x] = santa_filter[i,j] 
#                     #if santa_filter[i,j][3] != 0:
#             cv2.putText(img_copy, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
#             cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         else:
#             cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        
#     cv2.imshow('Emotion Detector', img_copy)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [3]:
#### WORKING CODE for neutral or not neural
# while True:
#     # Grab a single frame of video
#     ret, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_classifier.detectMultiScale(gray, 1.3, 5)
#     img_copy = np.copy(frame)
    
#     for (x, y, w, h) in faces:
#         roi_gray = gray[y:y + h, x:x + w]
#         roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        
#         # rect,face,image = face_detector(frame)

#         if np.sum([roi_gray]) != 0:
#             roi = roi_gray.astype('float') / 255.0
#             roi = img_to_array(roi)
#             roi = np.expand_dims(roi, axis=0)

#             # make a prediction on the ROI, then lookup the class

#             preds = classifier.predict(roi)[0]
#             label = class_labels[preds.argmax()]
#             label_position = (x, y)
            
#             img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)
            
#             # Beard filter
#             santa_filter = cv2.imread(emoji_folder + '/angry.png', -1)
#             santa_filter = cv2.resize( santa_filter, (w,h))
#             sw,sh,sc = santa_filter.shape
            
#             if label == 'Neutral':
#                 for i in range(0,sw):       # Overlay the filter based on the alpha channel
#                     for j in range(0,sh):
#                         img_copy[i+y,j+x] = santa_filter[i,j] 
            
#                     #if santa_filter[i,j][3] != 0:
#             cv2.putText(img_copy, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
#             cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         else:
#             cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        
#     cv2.imshow('Emotion Detector', img_copy)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [4]:
#### WORKING CODE for neutral or not neural OPTIMISED
# while True:
#     # Grab a single frame of video
#     ret, frame = cap.read()
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     faces = face_classifier.detectMultiScale(gray, 1.3, 5)
#     img_copy = np.copy(frame)
    
#     for (x, y, w, h) in faces:
#         roi_gray = gray[y:y + h, x:x + w]
#         roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        
#         # rect,face,image = face_detector(frame)

#         if np.sum([roi_gray]) != 0:
#             roi = roi_gray.astype('float') / 255.0
#             roi = img_to_array(roi)
#             roi = np.expand_dims(roi, axis=0)

#             # make a prediction on the ROI, then lookup the class

#             preds = classifier.predict(roi)[0]
#             label = class_labels[preds.argmax()]
#             label_position = (x, y)
            
#             img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)
            
#             #  filter   
#             if label == 'Neutral':
#                 filter_frame = cv2.resize( surprised_filter , (w,h))
#                 sw,sh,sc = filter_frame.shape
#                 for i in range(0,sw):       # Overlay the filter based on the alpha channel
#                     for j in range(0,sh):
#                         img_copy[i+y,j+x] = filter_frame[i,j] 
            
#                     #if santa_filter[i,j][3] != 0:
#             cv2.putText(img_copy, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
#             cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
#         else:
#             cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        
#     cv2.imshow('Emotion Detector', img_copy)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
        
# cap.release()
# cv2.destroyAllWindows()

In [1]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier(
    r'C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector-main\haarcascade.xml')
classifier = load_model(
    r'C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector-main\Emo_little-h5.h5')

class_labels = ['Angry', 'Happy', 'Neutral', 'Sad', 'Surprise']

emoji_folder = r'C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector-main\emoji'

In [2]:
cap = cv2.VideoCapture(0)

In [3]:
angry_filter = cv2.imread(emoji_folder + '/angry.png', -1)
happy_filter = cv2.imread(emoji_folder + '/happy.png', -1)
neutral_filter = cv2.imread(emoji_folder + '/neutral.png', -1)
sad_filter = cv2.imread(emoji_folder + '/sad.png', -1)
surprised_filter = cv2.imread(emoji_folder + '/surprised.png', -1)

In [4]:
#### WORKING CODE for neutral or not neural OPTIMISED
while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    img_copy = np.copy(frame)
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)
        
        # rect,face,image = face_detector(frame)

        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # make a prediction on the ROI, then lookup the class

            preds = classifier.predict(roi)[0]
            label = class_labels[preds.argmax()]
            label_position = (x, y)
            
            img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)
            
            #  filter   
            if label == 'Surprise':
                filter_frame = cv2.resize( surprised_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
                        
            if label == 'Happy':
                filter_frame = cv2.resize( happy_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
                        
            if label == 'Angry':
                filter_frame = cv2.resize( angry_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
                        
            if label == 'Sad':
                filter_frame = cv2.resize( sad_filter , (w,h))
                sw,sh,sc = filter_frame.shape
                for i in range(0,sw):       # Overlay the filter based on the alpha channel
                    for j in range(0,sh):
                        img_copy[i+y,j+x] = filter_frame[i,j]
            
                    #if santa_filter[i,j][3] != 0:
            cv2.putText(img_copy, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
        else:
            cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        
    cv2.imshow('Emotion Detector', img_copy)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [1]:
### neutral image not working
import cv2
emoji_folder = r'C:\Users\Pallavi Shete\Documents\Emotion_Detector-main\Emotion_Detector-main\emoji'
santa_filter = cv2.imread(emoji_folder + '/surprised.png', -1)
santa_filter = cv2.resize( santa_filter, (100,100))
cv2.imshow('Emotion', santa_filter)
cv2.waitKey(5000)
cv2.destroyAllWindows()
# santa_filter = cv2.resize( santa_filter, (100,100))
sw,sh,sc = santa_filter.shape

In [ ]:
#### Emojification #####


In [ ]:
# Implement the model in real-time 

# Importing the libraries
import numpy as np
from training import get_model, load_trained_model, compile_model
import cv2

# Load the trained model
model = get_model()
compile_model(model)
load_trained_model(model)

# Get frontal face haar cascade
#face_classifie = cv2.CascadeClassifier('C:\Users\Synergiz\PycharmProjects\OPEN_Emotion_Rec\haarcascade.xml')

# Get webcam
camera = cv2.VideoCapture(0)

# Run the program infinitely
while True:
    grab_trueorfalse, img = camera.read()       # Read data from the webcam
    
    # Preprocess input fram webcam
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)        # Convert RGB data to Grayscale
    faces = face_classifie.detectMultiScale(gray, 1.3, 5)     # Identify faces in the webcam
    
    # For each detected face using tha Haar cascade
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        img_copy = np.copy(img)
        img_copy_1 = np.copy(img)
        roi_color = img_copy_1[y:y+h, x:x+w]
        
        width_original = roi_gray.shape[1]      # Width of region where face is detected
        height_original = roi_gray.shape[0]     # Height of region where face is detected
        img_gray = cv2.resize(roi_gray, (48, 48))       # Resize image to size 96x96
        img_gray = img_gray/255         # Normalize the image data
        
        img_model = np.reshape(img_gray, (1,48,48,1))   # Model takes input of shape = [batch_size, height, width, no. of channels]
        keypoints = model.predict(img_model)[0]         # Predict keypoints for the current input
        
        # Keypoints are saved as (x1, y1, x2, y2, ......)
        x_coords = keypoints[0::2]      # Read alternate elements starting from index 0
        y_coords = keypoints[1::2]      # Read alternate elements starting from index 1
        
        x_coords_denormalized = (x_coords+0.5)*width_original       # Denormalize x-coordinate
        y_coords_denormalized = (y_coords+0.5)*height_original      # Denormalize y-coordinate
        
        for i in range(len(x_coords)):          # Plot the keypoints at the x and y coordinates
            cv2.circle(roi_color, (x_coords_denormalized[i], y_coords_denormalized[i]), 2, (255,255,0), -1)
        
      
    # Particular keypoints for scaling and positioning of the filter
        top = (int(x_coords_denormalized[11]), int(y_coords_denormalized[33]))
        bottom = (int(x_coords_denormalized[3]), int(y_coords_denormalized[5]))
       
        
        
        # Scale filter according to keypoint coordinates
        smiley = top[0] - bottom[0]
       
        
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGR2BGRA)       # Used for transparency overlay of filter using the alpha channel

        # Beard filter
        face = cv2.imread('emoji/happy.png', -1)
        face = cv2.resize(face, (smiley*3,150))
        sw,sh,sc = santa_filter.shape
        
        for i in range(0,sw):       # Overlay the filter based on the alpha channel
            for j in range(0,sh):
                if np.sum([smiley]) != 0:
                    smiley = smiley.astype('float') / 255.0
                    smiley = img_to_array(smiley)
                    smiley = np.expand_dims(smiley, axis=0)
                if smiley[i,j][3] != 0:
                    img_copy[top[1]+i+y-20, bottom[0]+j+x-60] = face[i,j]
                    
      
        
        img_copy = cv2.cvtColor(img_copy, cv2.COLOR_BGRA2BGR)       # Revert back to BGR
        
        cv2.imshow('Output',img_copy)           # Output with the filter placed on the face
        cv2.imshow('Keypoints predicted',img_copy)        # Place keypoints on the webcam input
        
    cv2.imshow('Webcam',img)        # Original webcame Input
    
    if cv2.waitKey(1) & 0xFF == ord("q"):   # If 'q' is pressed, stop reading and break the loop
        break